# RGBs 和其他复合体
气象卫星仪器波段可以以不同波长观察地球。每个波段代表在某一特定波长或波长范围所观测到的电磁辐射。虽然这种方法非常有用且灵活，且可视化所需处理量极少，但需要观测者了解每个波段或波长所能提供的信息。除了单独查看各个波段外，查看卫星数据的另一种常见方式是将多个波段合成一张图像，如RGB（红、绿、蓝）图像。通过将特定波段分配到输出RGB图像中的特定通道，我们可以提供一种简便的方式来识别数据中的特征。这样，我们无需了解特定波长对什么敏感，而是可以专注于特定RGB组合中颜色所代表的含义。
使用我们之前提供的ABI CONUS数据，让我们看看Satpy提供的部分内置RGB合成图像。

In [ ]:
%run ../init_notebook.py
from satpy import Scene
from glob import glob

filenames = glob('../data/abi_l1b/20180511_texas_fire_abi_l1b_conus/*.nc')
scn = Scene(reader='abi_l1b', filenames=filenames)
scn.available_composite_names()

上述列表显示了Satpy能够生成的合成图像及其所需的输入数据。Satpy中的许多合成方法都来自于EUMETSAT、NOAA或NASA等科学机构的标准配方。首先来看一下`airmass` RGB。`airmass` RGB由以下波段组成：
```
R: C08 - C10
G: C12 - C13
B: C08
```

红色通道是C08（6.185微米）和C10（7.34微米）波段的差值，绿色通道是C12（9.61微米）和C13（10.35微米）波段的差值，蓝色通道是C08（6.185微米）波段。
EUMETSAT EUMeTrain 手册总结了 `airmass` RGB 的使用方法：
> 由于本合成图包含了两个水汽通道（WV 6.2 µm 和 WV 7.3 µm），其主要应用是检测动态过程，如快速气旋生成、急流和位涡异常。
了解更多 [EUMeTrain 网站](http://www.eumetrain.org/data/3/306/navmenu.php?tab=9&page=4.0.0)。
在 Satpy 中，我们可以像加载任何普通的文件提供的产品一样加载预配置的合成数据。Satpy 会查看该配置方案，确定需要加载哪些波段，需要应用哪些修改/校正以满足该配置方案，然后返回一个表示该 RGB 的 `DataArray`。

In [ ]:
scn.load(['airmass'])
scn['airmass']

上述xarray输出的关键部分是，这是我们第一次在Satpy中处理具有`(bands: 3, y: 1500, x: 2500)`维度的三维DataArray。`bands`维度告诉我们每个“波段”代表什么。在此情况下，表示RGB图像的红、绿、蓝通道。

In [ ]:
scn['airmass'].coords['bands']

让我们将这个产品保存为PNG图像，以便了解`airmass` RGB的样子。

In [ ]:
from dask.diagnostics import ProgressBar
with ProgressBar():
    scn.save_datasets(writer='simple_image')

我们可以在图像中看到各种波段以不同方式结合，某些颜色可以代表地球的特定状态。
在本教程的前面部分，我们讨论了“增强”数据的过程，即将观测到的科学数据值缩放到图像值（例如 0-255 的整数）。这一过程由 `save_datasets` 自动完成。如果我们想在 matplotlib 图像中显示数据，则需要手动进行此操作。Satpy 提供了一个实用函数 `get_enhanced_image` 来简化这一过程。
我们可以告诉 xarray 哪个维度代表我们的 `DataArray` 的“RGB”轴。我们通过传递带有维度名称的 `rgb` 关键字参数来实现这一点。在 Satpy 中，这个维度应始终称为 `'bands'`。否则 matplotlib 将无法正确绘制我们的 RGB 图像。
最后，我们将告诉 `imshow` 方法我们 *知道* 我们的 RGB 值将在 0 到 1 之间，通过 `vmin` 和 `vmax` 关键字参数，因此无需计算数据的最小值/最大值。
让我们绘制我们的 `airmass` RGB 合成图。

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
from satpy.writers import get_enhanced_image

plt.figure()
img = get_enhanced_image(scn['airmass'])
# get DataArray out of `XRImage` object
img_data = img.data
img_data.plot.imshow(rgb='bands', vmin=0, vmax=1)

## 合成不同分辨率
在生成 `airmass` RGB 时，我们很幸运，所有输入数据的分辨率都相同。现在来看 `natural_color` RGB，其中输入分辨率不同。如 EUMeTrain 所述，`natural_color` RGB 在观察雪和冰时非常有用：
> 地表积雪以及山地冰盖、结冰的湖泊和海冰在自然色RGB图像中呈现青色。
有关更多信息，请参见[EUMeTrain 网站](http://www.eumetrain.org/resources/MSG_natcolour_cyan_snow.html)。
对于ABI仪器，这种合成图是由以下通道组合而成：
```
R: C05 (1000m)
G: C03 (1000m)
B: C02 (500m)
```

这意味着我们正在尝试组合不同形状的数组。我们唯一知道的使数据具有相同分辨率并使其更容易比较或组合的方法，就是进行重采样。让我们加载这个合成数据，并看看Satpy如何处理生成此类合成图像。

In [ ]:
scn.load(['natural_color'])

In [ ]:
'natural_color' in scn

In [ ]:
scn.missing_datasets

我们并没有做错任何事情，但 Satpy 的 `Scene` 中并没有 `natural_color` 合成图像。Satpy 知道它无法生成该合成图像的原因是数据的分辨率不一致。`Scene` 知道我们需要先进行重采样，才能生成该合成图像。现在我们使用 `native` 重采样器来完成这一操作。我们还将使用 `min_area` 方法将数据聚合到 `Scene` 中的最低分辨率；在此情况下为 2km（来自 `airmass` 依赖项）：

In [ ]:
new_scn = scn.resample(scn.min_area(), resampler='native')
new_scn['natural_color']

一旦重采样完成，`Scene` 将尝试重新生成之前请求的任何复合图像。一旦成功生成，`natural_color` RGB 就会像其他产品一样显示出来。让我们按照之前用于绘制 `airmass` RGB 的相同步骤，但这次用于 `natural_color` RGB。请注意，我们使用的是 `new_scn`，因为 `natural_color` RGB 仅存在于这个重采样的 `Scene` 中。

In [ ]:
with ProgressBar():
    plt.figure()
    img = get_enhanced_image(new_scn['natural_color'])
    # get DataArray out of `XRImage` object
    img_data = img.data
    img_data.plot.imshow(vmin=0, vmax=1, rgb='bands')

## 自定义 RGB
现在我们已经学习了如何查看可用的内置组合、加载它们以及生成它们。让我们来创建自己的自定义RGB。让我们重新创建之前的场景，并加载三个希望放入RGB中的通道。

In [ ]:
filenames = glob('../data/abi_l1b/20180511_texas_fire_abi_l1b_conus/*.nc')
scn = Scene(reader='abi_l1b', filenames=filenames)
scn.available_dataset_names()

In [ ]:
r_band = 'EDITME'
g_band = 'EDITME'
b_band = 'EDITME'
scn.load([r_band, g_band, b_band])

我们的 `Scene` 对象现在已加载了三个通道。我们现在将使用来自 Satpy 内置合成的特殊 `GenericCompositor` 来帮我们合并这些数组并生成一个 RGB。让我们看看尝试这样做时会发生什么：

In [ ]:
from satpy.composites import GenericCompositor
comp = GenericCompositor('my_rgb')
my_rgb = comp((scn[r_band], scn[g_band], scn[b_band]))

要求：
- 保留 Markdown 格式
- 保留代码、URL 和 HTML 标签
- 使用标准技术术语
- 仅输出翻译，不加解释

待翻译文本：
如果你选择的通道具有不同的分辨率，你应该会从上述操作中收到一个 `IncompatibleAreas` 异常。就像之前的 `natural_color` 一样，这是 Satpy 的合成器在告诉我们，我们的输入具有不同的分辨率，需要进行重采样。
让我们使用 `native` 重采样器进行重采样以修复分辨率，并尝试使用新重采样的 `Scene` 重新生成我们的合成图像。
即使您未收到错误信息，请执行下面的步骤。这并非必要，但会使后续步骤更简单。

In [ ]:
new_scn = scn.resample(resampler='native')
my_rgb = comp((new_scn[r_band], new_scn[g_band], new_scn[b_band]))
my_rgb

我们成功生成了一个RGB `DataArray`对象。让我们来绘制它并看看它的样子。

In [ ]:
plt.figure()
img = get_enhanced_image(my_rgb)
# get DataArray out of `XRImage` object
img_data = img.data
img_data.plot.imshow(vmin=0, vmax=1, rgb='bands')

上述图像可能看起来有点奇怪（颜色非常鲜艳，颜色非常暗淡）。这是因为除非另有配置，Satpy会将RGB值缩放至它们的最小值和最大值之间，但通常这不是我们想要的。我们告诉xarray和matplotlib我们的RGB数据在0到1之间，但根据你选择的通道，实际值要高得多（例如，反射率是0-120%，亮度温度是200-250K）。
### 练习:
**时间: 10-15 分钟**
为了找到使我们的数据看起来最佳的范围，让我们对下方的数据进行线性缩放。我们将分别对每个波段进行缩放，其中第一个数字代表我们的最小值，第二个代表最大值，第三个再次代表最小值。最终结果应该是一个RGB `DataArray`，其中我们数据的主要部分位于0到1之间。请记住我们正在缩放的数据类型：反射率或亮温。
记住生成最佳图像的参数。我们将在下一节中使用这些参数。

In [ ]:
r_norm = (new_scn[r_band] - 220) / (260 - 220)
g_norm = (new_scn[g_band] - 220) / (260 - 220)
b_norm = (new_scn[b_band] - 0) / (120 - 0)
my_rgb = comp((r_norm, g_norm, b_norm))

plt.figure()
my_rgb.plot.imshow(vmin=0, vmax=1, rgb='bands')

## 配置复合配方
既然我们已经找到了喜欢的RGB配方，可能以后想再次使用它。我们可以编写一个生成该配方的Python脚本或模块，但这要求我们在从Scene加载时知道哪些波段用于RGB，并且会使合成操作更加复杂。
Satpy 最初是为帮助自动和重复的操作处理而构建的。为了实现这一点，Satpy 使用了一系列内置在包中的 YAML 配置文件，但用户也可以对其进行自定义。本教程附带了用于本次练习的模板 YAML 文件，我们现在将对其进行填写。
1. 添加复合配方
要将波段组合添加到 YAML 配置文件中，请首先导航到 `notebooks/composites/` 目录 [此处](/tree/notebooks/composites)，然后在文本编辑器中打开 `abi.yaml` 文件。或者，您可以使用 Jupyter Notebook 的编辑器，通过访问 [abi.yaml](/edit/notebooks/composites/abi.yaml) 进行操作。我们希望将以下文本添加到此文件中：   
   ```yaml
   composites:
     my_rgb:
       compositor: !!python/name:satpy.composites.GenericCompositor
       prerequisites:
       - name: YOUR_R_CHANNEL
       - name: YOUR_G_CHANNEL
       - name: YOUR_B_CHANNEL
    ```
    
由于这个 `composites` 目录位于我们的当前目录中，Satpy 会找到它，并将我们的合成图像作为可加载的内容包含进去。我们告诉它我们的合成图像名称为 `my_rgb`，希望使用 `GenericCompositor` 类，并且它依赖于我们指定的三个通道。    
2. 添加增强配方
要添加我们的颜色限制，我们将编辑位于 `notebooks/enhancements/abi.yaml` 中的一个独立文件。点击[此处](/edit/notebooks/enhancements/abi.yaml)使用 Jupyter Notebook 的编辑器来编辑该文件。我们希望增强功能呈现如下效果：   
   ```yaml
   enhancements:
     my_rgb:
       name: my_rgb
       operations:
       - name: color_limits
         method: !!python/name:satpy.enhancements.stretch
         kwargs: {stretch: 'crude', min_stretch: [RMIN, GMIN, BMIN], max_stretch: [RMAX, GMAX, BMAX]}
   ```
   
这将使用内置的 `stretch` 方法，在我们指定的最小值和最大值之间进行简单的线性缩放。你应该将 `XMIN` 和 `XMAX` 的值替换为你在上述练习中得出的值。   

现在我们已经配置了合成数据的创建和缩放，就可以像Satpy中的其他合成数据一样加载它。请注意，我们不需要显式预加载输入通道，因为配方会告诉`Scene`需要什么。

In [ ]:
filenames = glob('../data/abi_l1b/20180511_texas_fire_abi_l1b_conus/*.nc')
scn = Scene(reader='abi_l1b', filenames=filenames)
scn.available_composite_names()

In [ ]:
scn.load(['my_rgb'])

就像之前一样，如果我们的合成需要多种分辨率，那么我们需要进行重采样以使所有输入大小相同。

In [ ]:
new_scn = scn.resample(resampler='native')
new_scn['my_rgb']

我们成功生成了RGB，现在来绘制它。我们将像本课开始时那样使用`get_enhanced_image`函数来应用我们配置的增强处理。

In [ ]:
plt.figure()
# get DataArray out of `XRImage` object with .data
img_data = get_enhanced_image(new_scn['my_rgb']).data
img_data.plot.imshow(vmin=0, vmax=1, rgb='bands')

现在我们已从预配置的YAML文件中加载了合成数据，并像最初使用的`airmass`和`natural_color` RGB一样将其显示出来。我们也可以使用之前学到的其他方法，比如`crop`或`save_datasets`，来进行其他操作。在接下来的几节课中，我们将继续使用这个RGB，并展示如何通过复用配置的方案使我们的工作更加简便。